**Main Game**

In [ ]:
import pygame
import random
import sys

# Initialize Pygame
pygame.init()

# Set up display
width, height = 800, 600
screen = pygame.display.set_mode((width, height))
pygame.display.set_caption("Geometry Slide")

# Colors
white = (255, 255, 255)
black = (0, 0, 0)

# Player settings
player_size = 50
player_x = width // 2
player_y = 400
player_speed = 5
player = pygame.Rect(player_x, player_y, player_size, player_size)
jump_strength = 25
can_jump = False
coyote_time = 0

# Platform settings
platform_start_y = 200
platform_width = 90
platform_height = 40

platform_1 = pygame.Rect((width+platform_width)/3, platform_start_y, platform_width, platform_height)
platform_2 = pygame.Rect(2*(width+platform_width)/3, platform_start_y, platform_width, platform_height)
platform_3 = pygame.Rect((width+platform_width), platform_start_y, platform_width, platform_height)
platforms = [platform_1, platform_2, platform_3]

items = []

start_platform_left = False
start_platform = pygame.Rect(300, 500, 200, 20)

platform_velocity = 2

last_y = platform_start_y

# Gravity settings
gravity = 0.9816
player_velocity_y = 0
max_fall_speed = 100

#Create score
score = 0
font = pygame.font.Font(None, 36)
last_platform = None

# Create a clock object to control the frame rate
clock = pygame.time.Clock()
fps = 60  # Set the desired frames per second

background_image = pygame.image.load('Background_Sky.jpg')  # Load the background image
background_image = pygame.transform.scale(background_image, (width, height))  # Scale it to fit the window

character_screen = pygame.image.load('Character_Screen.jpg')  # Load the background image
character_screen = pygame.transform.scale(character_screen, (width, height))  # Scale it to fit the window

platform_image = pygame.image.load('Platform_Cloud.png')  # Load the platform image
platform_image = pygame.transform.scale(platform_image, (platform_width, platform_height))  # Scale it to fit the platform

# Main loop
running = True
character_chosen = False
while running:
    while not character_chosen:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                character_chosen = True
                running = False
        keys = pygame.key.get_pressed()
        if keys[pygame.K_1]:
            player_image = pygame.image.load('Player_Anton.png')  # Load the player image
            player_image = pygame.transform.scale(player_image, (player_size, player_size))
            player_color = (157, 217, 243)
            character_chosen = True
        if keys[pygame.K_2]:
            player_image = pygame.image.load('Player_Rasmus.png')  # Load the player image
            player_image = pygame.transform.scale(player_image, (player_size, player_size))
            player_color = (246, 211, 189)
            character_chosen = True
        if keys[pygame.K_3]:
            player_image = pygame.image.load('Player_Casper.png')  # Load the player image
            player_image = pygame.transform.scale(player_image, (player_size, player_size))
            player_color = (21, 79, 48)
            character_chosen = True
        if keys[pygame.K_4]:
            player_image = pygame.image.load('Player_Torben.png')  # Load the player image
            player_image = pygame.transform.scale(player_image, (player_size, player_size))
            player_color = (237, 232, 208)
            character_chosen = True
        if keys[pygame.K_5]:
            player_image = pygame.image.load('Player_Rektor.png')  # Load the player image
            player_image = pygame.transform.scale(player_image, (player_size, player_size))
            player_color = (229, 228, 226)
            character_chosen = True
        if keys[pygame.K_6]:
            player_image = pygame.image.load('Player_Henrik.png')  # Load the player image
            player_image = pygame.transform.scale(player_image, (player_size, player_size))
            player_color = (83, 96, 253)
            character_chosen = True
        if keys[pygame.K_7]:
            player_image = pygame.image.load('Player_Yoshi.png')  # Load the player image
            player_image = pygame.transform.scale(player_image, (player_size, player_size))
            player_color = (0, 255, 0)
            character_chosen = True
        if keys[pygame.K_8]:
            player_image = pygame.image.load('Player_Julemanden.png')  # Load the player image
            player_image = pygame.transform.scale(player_image, (player_size, player_size))
            player_color = (255, 0, 0)
            character_chosen = True
        if keys[pygame.K_9]:
            player_image = pygame.image.load('Player_Steve.png')  # Load the player image
            player_image = pygame.transform.scale(player_image, (player_size, player_size))
            player_color = (14, 174, 174)
            character_chosen = True

        screen.blit(character_screen, (0, 0))
        pygame.display.flip()

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
    # Fill the screen with white
    screen.blit(background_image, (0, 0))

    # Player movement
    keys = pygame.key.get_pressed()
    if keys[pygame.K_a]:
        player.x -= player_speed
    if keys[pygame.K_d]:
        player.x += player_speed


    # Check for jump
    coyote_time -= fps/20
    if keys[pygame.K_SPACE] and can_jump and coyote_time >= 0:
        player_velocity_y = -jump_strength
        can_jump = False
        if start_platform_left == False:
            start_platform_left = True

    # Apply gravity
    player_velocity_y += gravity
    if player_velocity_y > max_fall_speed:
        player_velocity_y = max_fall_speed

    player.y += player_velocity_y
    if player.y >= height + player_size + 10:
        running = False
    
    if player.y <= -player_size:
       pygame.draw.polygon(screen, black, ((player.x+0.5*player_size,23),(player.x+0.5*player_size-7,3),(player.x+0.5*player_size+7,3))) 
       pygame.draw.polygon(screen, player_color, ((player.x+0.5*player_size,20),(player.x+0.5*player_size-5,5),(player.x+0.5*player_size+5,5))) 

    # Collision detection with platforms

    if player.colliderect(start_platform) and player_velocity_y > 0 and not start_platform_left:
            player.y = start_platform.y - player.height
            player_velocity_y = 0
            can_jump = True
            coyote_time = 0.5*fps

    for platform in platforms:
        if player.colliderect(platform) and player_velocity_y > 0:
            #Player collision mechanics
            player.y = platform.y - player.height
            player_velocity_y = 0
            player.x -= platform_velocity

            #Jump mechanics
            can_jump = True
            coyote_time = 0.5*fps

            if platform != last_platform:
                score += 1
                last_platform = platform

        # Platform mpvement and spawn
        platform.x -= platform_velocity
        if platform.x <= -platform_width:
            if start_platform_left == False:
                platform.x = width
            else:
                platform.x = width + random.randint(0, 20)
                max_jump_height = jump_strength**2 / (2 * gravity)
                lower_limit = int(max(last_y - max_jump_height + 1, platform_height + player_size))
                platform.y = random.randint(lower_limit, height-platform_height)

                last_y = platform.y
            if len(items) < 3:
        # Draw platform images
    for platform in platforms:
        screen.blit(platform_image, (platform.x, platform.y))

    if not start_platform_left:
        pygame.draw.rect(screen, black, start_platform)
        message = "Jump to start the game"
        score_text = font.render(message, True, black)
        text_width, text_height = font.size(message)
        screen.blit(score_text, (width/2-text_width/2, height/2-text_height/2))
    else:
        platform_velocity += 0.001

    screen.blit(player_image, (player.x, player.y))

    score_text = font.render(f"Score: {score}", True, black)
    screen.blit(score_text, (10, 10))

    # Update the display
    pygame.display.flip()

    # Cap the frame rate to 60 fps
    clock.tick(fps)

# Quit Pygame
pygame.quit()
sys.exit()


SystemExit: 

**OOC** Object oriented code  **NEW**

In [25]:
import pygame
import random
import sys

class Game:
    def __init__(self):
        pygame.init()
        self.width, self.height = 800, 600
        self.screen = pygame.display.set_mode((self.width, self.height))
        pygame.display.set_caption("Cloud Jumper")

        self.white = (255, 255, 255)
        self.black = (0, 0, 0)

        self.player_size = 50
        self.player_speed = 5
        self.jump_strength = 25
        self.gravity = 0.9816
        self.max_fall_speed = 100

        self.platform_start_y = 200
        self.platform_width = 90
        self.platform_height = 40
        self.platform_velocity = 2

        self.item_size = 30

        self.score = 0
        self.font = pygame.font.Font(None, 36)
        self.last_platform = None

        self.clock = pygame.time.Clock()
        self.fps = 60

        self.background_image = pygame.image.load('Background_Sky.jpg')
        self.background_image = pygame.transform.scale(self.background_image, (self.width, self.height))

        self.character_screen = pygame.image.load('Character_Screen.jpg')
        self.character_screen = pygame.transform.scale(self.character_screen, (self.width, self.height))

        self.platform_image = pygame.image.load('Platform_Cloud.png')
        self.platform_image = pygame.transform.scale(self.platform_image, (self.platform_width, self.platform_height))

        self.item_image = pygame.image.load('Clock_item.png')
        self.item_image = pygame.transform.scale(self.item_image, (self.item_size, self.item_size))
        
        self.reverse_frames = 0
        self.reversing = False

        self.running = True
        self.character_chosen = False

        self.player = pygame.Rect(self.width // 2, 400, self.player_size, self.player_size)
        self.player_velocity_y = 0
        self.can_jump = False
        self.coyote_time = 0

        self.start_platform_left = False
        self.start_platform = pygame.Rect(300, 500, 200, 20)

        self.platforms = [
            pygame.Rect((self.width + self.platform_width) / 3, self.platform_start_y, self.platform_width, self.platform_height),
            pygame.Rect(2 * (self.width + self.platform_width) / 3, self.platform_start_y, self.platform_width, self.platform_height),
            pygame.Rect((self.width + self.platform_width), self.platform_start_y, self.platform_width, self.platform_height)
        ]

        self.items = []

        self.last_y = self.platform_start_y

    def choose_character(self):
        while not self.character_chosen:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    self.character_chosen = True
                    self.running = False
            keys = pygame.key.get_pressed()
            player_choices = {
                pygame.K_1: ('Player_Anton.png', (157, 217, 243)),
                pygame.K_2: ('Player_Rasmus.png', (246, 211, 189)),
                pygame.K_3: ('Player_Casper.png', (21, 79, 48)),
                pygame.K_4: ('Player_Torben.png', (237, 232, 208)),
                pygame.K_5: ('Player_Rektor.png', (229, 228, 226)),
                pygame.K_6: ('Player_Henrik.png', (83, 96, 253)),
                pygame.K_7: ('Player_Yoshi.png', (0, 255, 0)),
                pygame.K_8: ('Player_Julemanden.png', (255, 0, 0)),
                pygame.K_9: ('Player_Steve.png', (14, 174, 174))
            }

            for key, (image, color) in player_choices.items():
                if keys[key]:
                    self.player_image = pygame.image.load(image)
                    self.player_image = pygame.transform.scale(self.player_image, (self.player_size, self.player_size))
                    self.player_color = color
                    self.character_chosen = True

            self.screen.blit(self.character_screen, (0, 0))
            pygame.display.flip()

    def run_game(self):
        while self.running:
            self.choose_character()

            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    self.running = False

            self.screen.blit(self.background_image, (0, 0))

            keys = pygame.key.get_pressed()
            if keys[pygame.K_a]:
                self.player.x -= self.player_speed
            if keys[pygame.K_d]:
                self.player.x += self.player_speed

            self.coyote_time -= self.fps / 20
            if keys[pygame.K_SPACE] and self.can_jump and self.coyote_time >= 0:
                self.player_velocity_y = -self.jump_strength
                self.can_jump = False
                if not self.start_platform_left:
                    self.start_platform_left = True

            self.apply_gravity()

            if self.player.y >= self.height + self.player_size + 10:
                self.running = False

            if self.player.y <= -self.player_size:
                self.draw_player_above_screen()

            self.check_platform_collision()

            self.draw_platforms()
            self.draw_items()

            if not self.start_platform_left:
                self.draw_start_platform_message()
            else:
                self.platform_velocity += 0.001

            if self.reversing:
                self.reverse_frames -= 1
                if self.reverse_frames <= 0:
                    self.reversing = False
                    self.platform_velocity = -self.platform_velocity

            self.screen.blit(self.player_image, (self.player.x, self.player.y))

            self.draw_score()

            pygame.display.flip()

            self.clock.tick(self.fps)

        pygame.quit()
        sys.exit()

    def apply_gravity(self):
        self.player_velocity_y += self.gravity
        if self.player_velocity_y > self.max_fall_speed:
            self.player_velocity_y = self.max_fall_speed
        self.player.y += self.player_velocity_y

    def draw_player_above_screen(self):
        pygame.draw.polygon(self.screen, self.black, ((self.player.x + 0.5 * self.player_size, 23),
                                                      (self.player.x + 0.5 * self.player_size - 7, 3),
                                                      (self.player.x + 0.5 * self.player_size + 7, 3)))
        pygame.draw.polygon(self.screen, self.player_color, ((self.player.x + 0.5 * self.player_size, 20),
                                                             (self.player.x + 0.5 * self.player_size - 5, 5),
                                                             (self.player.x + 0.5 * self.player_size + 5, 5)))

    def check_platform_collision(self):
        if self.player.colliderect(self.start_platform) and self.player_velocity_y > 0 and not self.start_platform_left:
            self.player.y = self.start_platform.y - self.player.height
            self.player_velocity_y = 0
            self.can_jump = True
            self.coyote_time = 0.5 * self.fps

        for platform in self.platforms:
            if self.player.colliderect(platform) and self.player_velocity_y > 0:
                self.player.y = platform.y - self.player.height
                self.player_velocity_y = 0
                self.player.x -= self.platform_velocity

                self.can_jump = True
                self.coyote_time = 0.5 * self.fps

                if platform != self.last_platform:
                    self.score += 1
                    self.last_platform = platform

            platform.x -= self.platform_velocity
            if platform.x <= -self.platform_width:
                if not self.start_platform_left:
                    platform.x = self.width
                else:
                    platform.x = self.width + random.randint(0, 20)
                    max_jump_height = self.jump_strength ** 2 / (2 * self.gravity)
                    lower_limit = int(max(self.last_y - max_jump_height + 1, self.platform_height + self.player_size))
                    platform.y = random.randint(lower_limit, self.height - self.platform_height)
                    self.last_y = platform.y
    
    def draw_items(self):
        if random.randint(1, 1000) <= 1:
            item_y = random.randint(0, self.height-self.item_size)
            self.items.append(pygame.Rect(self.width, item_y, self.item_size, self.item_size))

        for item in self.items:
            self.screen.blit(self.item_image, (item.x, item.y))
            item.x -= self.platform_velocity

        self.items = [item for item in self.items if item.x > -self.item_size]

        for item in self.items:
            if self.player.colliderect(item):
                self.items.remove(item)
                self.reverse_frames = 150
                self.reversing = True
                self.platform_velocity = -self.platform_velocity
     
    def draw_platforms(self):
        for platform in self.platforms:
            self.screen.blit(self.platform_image, (platform.x, platform.y))

    def draw_start_platform_message(self):
        pygame.draw.rect(self.screen, self.black, self.start_platform)
        message = "Jump to start the game"
        score_text = self.font.render(message, True, self.black)
        text_width, text_height = self.font.size(message)
        self.screen.blit(score_text, (self.width / 2 - text_width / 2, self.height / 2 - text_height / 2))

    def draw_score(self):
        score_text = self.font.render(f"Score: {self.score}", True, self.black)
        self.screen.blit(score_text, (10, 10))


if __name__ == "__main__":
    game = Game()
    game.run_game()


SystemExit: 